In [99]:
import numpy as np
import matplotlib.pyplot as plt

from numba import jit

In [100]:
def initial_system_config(N):
    # Define a configuração inicial do sistema
    # S rcebe valores aleatórios -1 ou 1
    S = np.random.randint(0,2,N)
    S = 2 * S - 1
    return S

In [101]:
def expos(beta):
    ex = np.zeros(5,dtype=np.float32)
    ex[0]=np.exp(8.0*beta)
    ex[1]=np.exp(4.0*beta)
    ex[2]=1.0
    ex[3]=np.exp(-4.0*beta)
    ex[4]=np.exp(-8.0*beta)
    return ex

In [102]:
@jit(nopython=True)
def vizinhos(N):
    # Define a tabela de vizinhos
    L=int(np.sqrt(N))
    viz = np.zeros((N,4),dtype=np.int16)
    for k in range(N):
        viz[k,0]=k+1
        if (k+1) % L == 0:
            viz[k,0] = k+1-L
        viz[k,1] = k+L
        if k > (N-L-1):
            viz[k,1] = k+L-N
        viz[k,2] = k-1
        if (k % L == 0):
            viz[k,2] = k+L-1
        viz[k,3] = k-L
        if k < L:
            viz[k,3] = k+N-L
    return viz

In [103]:
def energy(N, viz, S):
    E = 0.0
    for i in range(N):
        h = S[viz[i,0]] + S[viz[i,1]]
        E += -S[i]*h
    return E

In [104]:
def metroplis_ising(N, T, mc_steps):    
# Passos gerais do algoritmo
    beta = 1/T
    ex = expos(beta)
    S = initial_system_config(N)
    energy_system = []
    for _ in range(mc_steps):
        viz = vizinhos(N)
        r = np.random.uniform(0,1)
        spin = np.random.randint(0, N-1)
        h = S[viz[spin,0]] + S[viz[spin,1]] + S[viz[spin,2]] + S[viz[spin,3]] # soma dos vizinhos
        de = int(S[spin] * h * 0.5 + 2)
        P = ex[de]
        if r <= P:
            S[spin] = -S[spin]
        energy_system.append(energy(N, viz, S))
    
    return energy_system
# Temperatura está em unidades de j/kB, onde kb é a constante de Boltzmann

In [105]:
def _plot(values, mc_steps):
    for value in values:
        plt.plot(mc_steps, value)

    plt.xlabel('Monte Carlo Steps')
    plt.ylabel('Energy')
    plt.show()

In [106]:
def plot_random_systems_energy(systems, mc_steps=1000):
    mc_steps = np.arange(0, mc_steps, 1)
    _plot(systems, mc_steps)


In [107]:
def generate_systems(N_range, T_range, n_systems=10,  mc_steps=1000):
    systems = []
    for _ in range(n_systems):
        T = np.random.uniform(T_range[0], T_range[1])
        N = np.random.randint(N_range[0], N_range[1]) ** 2
        systems.append(metroplis_ising(N, T, mc_steps))
    return systems

In [108]:
systems = generate_systems([20, 100], [0.4, 3], 20)
plot_random_systems_energy(systems)